In [3]:
import numpy as np

class HMM:
    def __init__(self, Ann, Bnm, Pi, O):
        self.A = np.array(Ann, np.float) # 状态转移概率
        self.B = np.array(Bnm, np.float) # 观察转移概率
        self.Pi = np.array(Pi, np.float) # 初始概率
        self.O = np.array(O, np.float)   # 观测序列
        self.N = self.A.shape[0]         # 状态数量
        self.M = self.B.shape[1]         # 可观测状态数量
        
    def viterbi(self): # 维特比预测
        # given O,lambda .finding I

        T = len(self.O) # 序列长度
        I = np.zeros(T, np.float) #最有的最有路径
        
        delta = np.zeros((T, self.N), np.float) # 存储用于动态规划的
        psi = np.zeros((T, self.N), np.float) # 存储路径
        
        for i in range(self.N):
            delta[0, i] = self.Pi[i] * self.B[i, self.O[0]] # 初始化状态
            psi[0, i] = 0 # 路径节点初始化为0
            
        for t in range(1, T): # 对于每个观测，逐层的进行动态规划
            for i in range(self.N): # 对于每个状态，逐个计算
                delta[t, i] = self.B[i, self.O[t]] * np.array([
                    delta[t - 1, j] * self.A[j, i] for j in range(self.N)]).max() # 状态转移 计算最有概率
                psi[t, i] = np.array([
                    delta[t - 1, j] * self.A[j, i] for j in range(self.N)]).argmax() # 状态转移 计算最有路径节点
        
        P_T = delta[T - 1, :].max() # 最优解概率
        I[T_1] = delta[T - 1, :].argmax() #最优路径
        
        for t in range(T - 2, -1, -1):
            I[t] = psi[t + 1, I[t + 1]]
        
        return I
    
    def forward(self):
        T = len(self.O) # 序列长度
        alpha = np.zeros((T, self.N), np.float)
        
        for i in range(self.N):
            alpha[0, i] = self.Pi[i] * self.B[i, self.O[0]] # 初始化前向概率
        
        for t in range(T - 1):
            for i in range(self.N):
                sum = 0.0
                for j in range(self.N):
                    sum += alpha[t, j] * self.A[j, i]
                alpha[t + 1, i] = sum * self.B[i, self.O[t + 1]]
                
        sum = 0.0
        for i in range(self.N):
            sum += alpha[T - 1, i]
        
        return sum, alpha
    
    def backward(self):
        T = len(self.O)
        beta = np.zeros((T, self.N), np.float)
        
        for i in range(self.N):
            beta[T - 1, i] = 1.0
            
        for t in range(T - 2, -1, -1):
            for i in range(self.N):
                sum = 0.0
                for j in range(self.N):
                    sum += self.A[i, j] * self.B[j, self.O[t + 1]] * beta[t + 1, j]
                beta[t, i] = sum
        
        sum = 0.0
        for i in range(self.N):
            sum += self.Pi[i] * self.B[i, self.O[0]] * beta[0, i]
        
        return sum, beta
    
    def gamma(self, alpha, beta): # 计算伽马值 即 t时刻出于qi状态的概率
        T = len(self.O)
        gamma = np.zeros((T, self.N), np.float)
        
        for t in range(T):
            for i in range(self.N):
                gamma[t, i] = alpha[t, i] * beta[t, i] / sum(
                    alpha[t, j] * beta[t, j] for j in range(self.N)
                )
        
        return gamma
    
    def theta(self, alpha, beta):
        T = len(self.O)
        thetas = np.zeros((T - 1, self.N, self.N), np.float)
        for t in range(T - 1):
            for i in range(self.N):
                for j in range(self.N):
                    numerator = alpha[t,i] * self.A[i,j] * self.B[j,self.O[t+1]] * beta[t+1,j]
                    denominator = sum( 
                        sum(alpha[t,i1] * self.A[i1,j1] * self.B[j1,self.O[t+1]] * beta[t+1,j1]
                            for j1 in range(self.N)
                           ) for i1 in range(self.N)
                    )
                    thetas[t, i, j] = numerator / denominator
                    
        return thetas
                    
    def Baum_Welch(self):
        T = len(self.O)
        V = [k for k in range(self.M)]
        
        # 初始化初始参数
        self.A = np.array(([[0,1,0,0],[0.4,0,0.6,0],[0,0.4,0,0.6],[0,0,0.5,0.5]]), np.float)
        self.B = np.array(([[0.5,0.5],[0.3,0.7],[0.6,0.4],[0.8,0.2]]), np.float)
        self.Pi = np.array(([1.0 / self.N] * self.N), np.float)
        
        x = 1
        delta_lambda = x + 1
        times = 0
        
        while delta_lambda > x:
            Polambda1, alpha = self.forward()
            Polambda2, beta = self.backward()
            gamma = self.gamma(alpha,beta)
            theta = self.theta(alpha,beta)
            
            lambda_n = [self.A, self.B, self.Pi]
            
            for i in range(self.N):
                for j in range(self.N):
                    numerator = sum(theta[t,i,j] for t in range(T-1))
                    denominator = sum(gamma[t,i] for t in range(T-1))
                    self.A[i, j] = numerator / denominator
                    
            for j in range(self.N):
                for k in range(self.M):
                    numerator = sum(gamma[t,j] for t in range(T) if self.O[t] == V[k])
                    denominator = sum(gamma[t,j] for t in range(T))
                    self.B[i, k] = numerator / denominator
                    
            for i in range(self.N):
                self.Pi[i] = gamma[0,i]
                
            delta_A = map(abs, lambda_n[0] - self.A)
            delta_B = map(abs, lambda_n[1] - self.B)
            delta_Pi = map(abs, lambda_n[2] - self.Pi)
            delta_lambda = sum([
                sum(sum(delta_A)), 
                sum(sum(delta_B)), 
                sum(delta_Pi)
            ])

            times += 1
            print times
        
        return self.A, self.B, self.Pi